# 7.3 Dataframes en PySpark

In [2]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster('local').setAppName('Mi programa')
sc = SparkContext(conf = conf)

In [4]:
import pandas as pd
df = pd.read_csv('./base_datos_2008.csv',nrows=1e6)

In [5]:
df.head(2) # Mostrar las dos primeras filas

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,335,...,4.0,8.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN
1,2008,1,3,4,754.0,735,1002.0,1000,WN,3231,...,5.0,10.0,0,NaN,0,NaN,NaN,NaN,NaN,NaN


In [7]:
# Ver los datos de la manera anterior con PySpark
from pyspark.sql.types import StringType
from pyspark import SQLContext
# Importar un segundo contexto vinculado a SQL
sqlContext = SQLContext(sc)

# Se añaden opciones, por ejemplo: 
# - Que la primera fila son los nombres de las columnas
# - la segunda opción es que intente saber qué tipo de variables es cada uno (si no se agrega convierte todo en texto) con inferSchema
dfspark = sqlContext.read.format('csv').option('header','true').option('inferSchema','true').load('./base_datos_2008.csv')

In [8]:
# Motrar datos
# show es uno de los métodos para ver la salida
dfspark.show(2)

+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|2008|    1|         3|        4|   2003|      1955|   2211|      2225|           WN|      335

In [9]:
# Método recomendado para mostrar la salida
dfspark.head(2)
# Hay datos que los toma de manera erronea

[Row(Year=2008, Month=1, DayofMonth=3, DayOfWeek=4, DepTime='2003', CRSDepTime=1955, ArrTime='2211', CRSArrTime=2225, UniqueCarrier='WN', FlightNum=335, TailNum='N712SW', ActualElapsedTime='128', CRSElapsedTime='150', AirTime='116', ArrDelay='-14', DepDelay='8', Origin='IAD', Dest='TPA', Distance=810, TaxiIn='4', TaxiOut='8', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA'),
 Row(Year=2008, Month=1, DayofMonth=3, DayOfWeek=4, DepTime='754', CRSDepTime=735, ArrTime='1002', CRSArrTime=1000, UniqueCarrier='WN', FlightNum=3231, TailNum='N772SW', ActualElapsedTime='128', CRSElapsedTime='145', AirTime='113', ArrDelay='2', DepDelay='19', Origin='IAD', Dest='TPA', Distance=810, TaxiIn='5', TaxiOut='10', Cancelled=0, CancellationCode=None, Diverted=0, CarrierDelay='NA', WeatherDelay='NA', NASDelay='NA', SecurityDelay='NA', LateAircraftDelay='NA')]

In [10]:
# Indicar cuántas filas se han cargado
dfspark.count()

7009728

In [12]:
# Mostrar cómo se realizará algún tipo de transformación básia ne los datos en una fracción de ella
dfspark = dfspark.sample(fraction = 0.001, withReplacement = False)
# Sin reemplazo, al ser una tranformación se ejecuta al instante

In [14]:
# Ver cuántas filas se tienen actualmente
dfspark.count()
# Reduce el tamaño de la base de datos

9

In [16]:
# Transformar al formato de interes
dfspark = dfspark.withColumn('ArrDelay',dfspark['ArrDelay'].cast('integer'))

In [18]:
# Eliminar los datos faltantes de las columnas de interés
df2 = dfspark.na.drop(subset = ['ArrDelay','DepDelay','Distance'])

In [19]:
# Ver que no están almacenadas como nulls
df2 = df2.filter('ArrDelay is not NULL') # Que el [nombre de columna] no sea null

In [20]:
# Pedir cuańtos datos se tiene
df2.count()

9

In [21]:
# Indicar qué tipo de variable es cada columna
df2.printSchema()
# Indica que si puede ser un valor nulo o no

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: string (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: string (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: string (nullable = true)
 |-- CRSElapsedTime: string (nullable = true)
 |-- AirTime: string (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: string (nullable = true)
 |-- TaxiOut: string (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)
 |-- Ca

In [22]:
# Aplicar operaciones numéricas utilizando bibliotecas externas
import numpy as np
# Devolver toda la inforamción de la columna y sacar la media
media = np.mean(df2.select('ArrDelay').collect())


In [23]:
media

2.3333333333333335

In [24]:
# Número de particiones RDD donde nuestra base de datos está distribuida
df2.rdd.getNumPartitions() # Depende de la configuración del ordenados

6